In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
import psycopg2
from sodapy import Socrata

In [2]:
# def read_db_water():
#     with open('db_config.json') as file:
#         db_config = json.load(file)

#     engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5432/{db_config["dbname"]}')

#     water = pd.read_sql('SELECT * FROM water_table', engine)

#     print(water.head())

# read_db_water()


In [2]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5432/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)
    
    return water

In [3]:
from transform_dag import apply_transformations

def transform_water():
    
    water = read_db_water()
    
    water = apply_transformations(water)
    
    print(water.head())

# Llamar a la función principal para procesar los datos
transform_water()



2024-05-03 12:59:25,578 - INFO - Starting transformations on water data.
2024-05-03 12:59:25,815 - INFO - Standardized place names.
2024-05-03 12:59:25,846 - INFO - Scaled numerical columns.
2024-05-03 12:59:25,908 - INFO - Filtered top influential parameters.
2024-05-03 12:59:26,010 - INFO - Classified IRCA values into categories.
2024-05-03 12:59:28,655 - INFO - Categorized treatment data.
2024-05-03 12:59:32,809 - INFO - Critical Proportion.
2024-05-03 12:59:32,860 - INFO - Dropped unnecessary columns.
2024-05-03 12:59:32,861 - INFO - Column names standardized
2024-05-03 12:59:32,861 - INFO - Standardized column names.
2024-05-03 12:59:32,862 - INFO - All transformations applied successfully.


          año nombredepartamento  div_dpto nombremunicipio  divi_muni  \
0  2010-01-01            Bolívar        13        El Guamo      13248   
1  2010-01-01            Bolívar        13        El Guamo      13248   
2  2010-01-01            Bolívar        13        El Guamo      13248   
3  2010-01-01            Bolívar        13        El Guamo      13248   
4  2010-01-01            Bolívar        13        El Guamo      13248   

   ircaminimo  ircamaximo  ircapromedio nombreparametroanalisis2  \
0         0.0       100.0         37.32        Alcanilidad Total   
1         0.0       100.0         37.32                 Aluminio   
2         0.0       100.0         37.32                 Arsénico   
3         0.0       100.0         37.32                   Cadmio   
4         0.0       100.0         37.32                   Calcio   

   numeroparametrospromedio  is_top_15   rango_irca tratamientocategoría  \
0                         2       True  Riesgo alto  Tratamiento parcial   


In [ ]:
def extrat_api(dataset_id, output_filename, limit=2000):
    client = Socrata("www.datos.gov.co", None)
    
    results = client.get(dataset_id, limit=limit)
    
    api = pd.DataFrame.from_records(results)
    
    api.to_csv(output_filename, index=False)
    
    print("Archivo descargado con éxito.")

extrat_api("tcwu-r53g", "datos_descargados.csv")


2024-05-03 11:12:42,021 - WARNING - Requests made without an app_token will be subject to strict throttling limits.


Archivo descargado con éxito.


In [5]:
from transform_dag import transformations_api_data

def transform_api():
    
    api = extrat_api()
    
    api = transformations_api_data(api)
    
    print(api.head())

# Llamar a la función principal para procesar los datos
transform_api()


ImportError: cannot import name 'transformations_api_data' from 'transform_dag' (c:\Users\JSLV3\Documents\5to Semestre\ETL\Proyecto Water Quality\1ra parte\Water_project\ETL-Proyect\Airflow_water\transform_dag.py)